In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

In [ ]:
import hvplot.pandas

# Basic Imports and definitions 

In [ ]:
def infer_dtypes(row0):
    column_types = row0.to_dict()
    for k in column_types:
        if column_types[k].endswith('s'):
            column_types[k] = 'category'
        elif column_types[k].endswith('n'):
            column_types[k] = 'float'
        elif column_types[k].endswith('d'):
            column_types[k] = 'datetime64[ns]'
    return column_types

In [ ]:
def parse_param_table(data_url):
    dfcomments=pd.read_csv(data_url,sep='#',nrows=100)

    dfcomments=dfcomments[dfcomments.iloc[:,0].isna()].iloc[:,1]
    dfcomments

    param_table_begin=dfcomments[dfcomments.str.find('TS_ID       Parameter Description') > 0].index[0]
    #dfcomments.str.split('\s+',expand=True)

    dfparam=dfcomments.iloc[param_table_begin:]
    param_table_end=dfparam[dfparam.isna()].index[0]

    dfparams=dfcomments.iloc[param_table_begin:param_table_end]
    param_columns=dfparams.iloc[0:1].str.strip().str.split('\s+').iloc[0]
    col_index=[dfparams.iloc[0].find(col) for col in param_columns]
    
    df = pd.DataFrame(columns=param_columns)
    
    for i in range(len(col_index) - 1):
        df.iloc[:, i] = (dfparams.iloc[1:].str.slice(col_index[i], col_index[i + 1])).str.strip()
    df.iloc[:, len(col_index) - 1] = dfparams.iloc[1:].str.slice(col_index[len(col_index) - 1],).str.strip()
    return df.reset_index().drop(columns='index')

In [ ]:
from usgs_maps import nwis

# Stations List Query

In [ ]:
stations_list_file='https://waterservices.usgs.gov/nwis/site/?format=rdb,1.0&stateCd=ca&siteStatus=all'
#stations_list_file='../tests/data/ca-stations-list.rdb.txt' # cached the above url for speed
dfstations = nwis.read_rdb(stations_list_file)
dfstations.head()

# Single station with detailed info

In [ ]:
dfs = nwis.read_rdb('https://waterservices.usgs.gov/nwis/site/?format=rdb,1.0&sites=09423350&seriesCatalogOutput=true&siteStatus=all')
dfs.head(3)

# Site Type Query

In [ ]:
# site type code
site_type_url='https://help.waterdata.usgs.gov/code/site_tp_query?fmt=html'
dflist=pd.read_html(site_type_url)
assert len(dflist) == 1
site_type=dflist[0]

In [ ]:
site_type.columns

In [ ]:
dict(zip(site_type.columns,['category']*len(site_type.columns)))

In [ ]:
site_type_dtype_map = {'Site Tp Cd': 'category',
 'Site Tp Srt Nu': 'int',
 'Site Tp Vld Fg': 'category',
 'Site Tp Prim Fg': 'category',
 'Site Tp Nm': 'string',
 'Site Tp Ln': 'string',
 'Site Tp Ds': 'string'}

In [ ]:
site_type.astype(dtype=site_type_dtype_map)
site_type.head(3)

# Plot those with latitude/longitude info

In [ ]:
dfll=dfstations.astype({'dec_lat_va':'float64','dec_long_va':'float64'},errors='ignore').dropna(how='any').reset_index(drop=True)

In [ ]:
dfll['dec_lat_va']=pd.to_numeric(dfll['dec_lat_va'])

In [ ]:
dfll['dec_long_va']=pd.to_numeric(dfll['dec_long_va'])

In [ ]:
dfll.hvplot.points(x='dec_long_va',y='dec_lat_va',geo=True,tiles='OSM',alpha=0.5, hover_cols='all').opts(frame_width=500)

# Data Query for station (siteid)

In [ ]:
data_url='https://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=11455780&startDT=2021-10-01&endDT=2021-10-30&siteStatus=all'

In [ ]:
dfdata=pd.read_csv(data_url,sep='\t',comment='#')
column_types = infer_dtypes(dfdata.iloc[0])
dfdata=dfdata.iloc[1:].reset_index().astype(dtype=column_types)
dfdata.head(3)

In [ ]:
dfdata.set_index('datetime')['16223_00095'].hvplot()

In [ ]:
dfparams=nwis.parse_param_table(data_url)
dfparams

# Monthlyqueries for data

In [ ]:
nwis.get_station_data('11455780','2021-09-01','2021-09-30')

In [ ]:
nwis.get_station_data('11455780','2021-10-01','2021-10-31')

In [ ]:
dfsd = nwis.get_station_detailed_info(11455780)
dfsd.head()

In [ ]:
dfsd.begin_date.min(),dfsd.end_date.max()

In [ ]:
dffull=nwis.get_station_data(11455780,'1997-12-18','2021-11-02')

In [ ]:
dffull.info()

In [ ]:
def to_date_format(str):
    try:
        return pd.to_datetime(str).strftime('%Y-%m-%d')
    except:
        return ''


def to_datetime(dstr):
    if dstr == '':
        return pd.Timestamp.now()
    else:
        return pd.to_datetime(dstr)


def to_year(dstr):
    return to_datetime(dstr).year


def sort_times(start, end):
    stime = to_datetime(start)
    etime = to_datetime(end)
    if stime < etime:
        return to_date_format(stime), to_date_format(etime)
    else:
        return to_date_format(etime), to_date_format(stime)

In [ ]:
import dask.dataframe as dd


def read_station_data_dd(siteid, start, end):
    # make sure start and end are in the right order, start < order
    start, end = sort_times(start, end)
    start_year = to_year(start)
    end_year = to_year(end) + 1
    url = 'https://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites={siteid}&startDT={year}-01-01&endDT={year}-12-31&siteStatus=all'
    list_urls = [url.format(siteid=siteid, year=syear)
                 for syear in range(start_year, end_year)]
    ddf = dd.read_csv(list_urls, blocksize=None, sep='\t', comment='#')
    # parse_dates=['DATE TIME','OBS DATE'] # doesn't work so will have to read in as strings and convert later
    # dd.visualize(): shows parallel tasks which are executed below
    df = ddf.compute()
    df.index = pd.to_datetime(df['datetime'])
    df = df.drop(columns=['datetime'])
    return df